In [1]:
from icr.recourse import recourse
from sklearn.ensemble import RandomForestClassifier
from icr.causality.scms.generic import GenericSCM
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
from icr.causality import DirectedAcyclicGraph
import numpy as np

In [129]:
#Liste de features geradas no grafo causal
notears_selected = [
   "Suic_familia",
    "Capaz de tomar decisões importantes",
    "Estudante",
    "Hipocondriase",
    "Sentimentos_culpa",
    "Trabalho e interesses",
    'Dep_familia',
    'Alc_familia',
    'Capaz de desfrutar das coisas',
    'Droga',
    'Suicidio',
   # 'Ansiedade',
     ]
dataframe = pd.read_csv('C:\\Users\Cristiane\Documents\\UFMG\Arquivos_Dissertacao\Projeto\data\\final_novo.csv',
                        sep=';')

dataframe = dataframe.drop(['Chave'], axis=1)
dataframe = dataframe.fillna(-1)  #todo -1
dataframe['sexo'].replace({'M': 0, 'F': 1}, inplace=True)
df_suic = dataframe.copy()
df_suic.loc[df_suic.Estudante == 0.5, 'Estudante'] = 1.0
df_suic = df_suic.astype(float)
#df_suic['Suicidio']=np.where((df_suic['Suicidio'] >1),1,0) #todo
df_suic_notears = df_suic.copy()
df_suic_notears = df_suic_notears[notears_selected]

In [288]:
# Ajuste conforme as colunas relevantes do seu DataFrame
'''cols = df_suic_notears.columns[df_suic_notears.columns != 'Suicidio']  # Suicidio é o alvo
X = df_suic[cols]
y = df_suic['Suicidio']

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)'''

In [130]:
# Definir as variáveis e as dependências (dicionário de adjacências)
adj = {
    'Suic_familia': ['Suicidio'],
    'Capaz de tomar decisões importantes': ['Suicidio'],
    'Estudante': ['Suicidio'],
    'Hipocondriase': ['Suicidio'],
    'Sentimentos_culpa': ['Suicidio'],
    'Trabalho e interesses': ['Suicidio'],
    'Dep_familia': ['Suicidio'],
    'Alc_familia': ['Suicidio'],
    'Capaz de desfrutar das coisas': ['Suicidio'],
    'Droga': ['Suicidio'],
   # 'Ansiedade': ['Suicidio']
}

# Lista de variáveis
var_names = list(adj.keys()) + ['Suicidio']

# Criar a matriz de adjacência
adj_matrix = np.zeros((len(var_names), len(var_names)))
for parent, children in adj.items():
    for child in children:
        parent_idx = var_names.index(parent)
        child_idx = var_names.index(child)
        adj_matrix[parent_idx, child_idx] = 1

# Criar o DAG a partir da matriz de adjacência
dag = DirectedAcyclicGraph(adjacency_matrix=adj_matrix, var_names=var_names)

print("DAG criado com sucesso!")
print("Nomes das variáveis:", dag.var_names)
# Verifique o DAG
print("Matriz de Adjacência:")
#print(dag.adjacency_matrix)
from icr.causality.scms.generic import GenericSCM

# Definir funções geradoras para o SCM
def generate_suicidio(context):
    suicidio = (
        context.get("Suic_familia", 0) * 0.4 +
        context.get("Capaz de tomar decisões importantes", 0) * 0.3 +
        context.get("Ansiedade", 0) * 0.3
    )
    return round(suicidio)

# Outras funções geradoras (valores padrão como exemplo)
generator_functions = {
    "Suic_familia": lambda context: context.get("Suic_familia", 0),
    "Capaz de tomar decisões importantes": lambda context: context.get("Capaz de tomar decisões importantes", 1),
    "Estudante": lambda context: context.get("Estudante", 0),
    "Hipocondriase": lambda context: context.get("Hipocondriase", 0),
    "Sentimentos_culpa": lambda context: context.get("Sentimentos_culpa", 0),
    "Trabalho e interesses": lambda context: context.get("Trabalho e interesses", 0),
    "Dep_familia": lambda context: context.get("Dep_familia", 0),
    "Alc_familia": lambda context: context.get("Alc_familia", 0),
    "Capaz de desfrutar das coisas": lambda context: context.get("Capaz de desfrutar das coisas", 1),
    "Droga": lambda context: context.get("Droga", 0),
   # "Ansiedade": lambda context: context.get("Ansiedade", 0),
    "Suicidio": generate_suicidio,
}

DAG criado com sucesso!
Nomes das variáveis: ['Suic_familia' 'Capaz de tomar decisões importantes' 'Estudante'
 'Hipocondriase' 'Sentimentos_culpa' 'Trabalho e interesses' 'Dep_familia'
 'Alc_familia' 'Capaz de desfrutar das coisas' 'Droga' 'Suicidio']
Matriz de Adjacência:


In [5]:
# Criar o SCM
# Função para aplicar intervenção manualmente e calcular valores
'''def apply_intervention(scm, intervention):
    # Inicializar o contexto com as intervenções
    context = {**intervention}
    
    # Atualizar valores para todas as variáveis do SCM com base no grafo
    for var in scm.dag.var_names:
        if var not in context:
            if var in generator_functions:
                context[var] = generator_functions[var](context)
    
    return context

# Exemplo de intervenção
intervention = {'Suic_familia': 1}
updated_values = apply_intervention(scm, intervention)

# Converter os valores em DataFrame para facilitar a visualização
df_values = pd.DataFrame([updated_values])
print("Valores após a intervenção:")
print(df_values)'''

Valores após a intervenção:
   Suic_familia  Capaz de tomar decisões importantes  Estudante  \
0             1                                    1          0   

   Hipocondriase  Sentimentos_culpa  Trabalho e interesses  Dep_familia  \
0              0                  0                      0            0   

   Alc_familia  Capaz de desfrutar das coisas  Droga  Ansiedade  Suicidio  
0            0                              1      0          0         1  


In [131]:
scm = GenericSCM(dag)
# Ajuste conforme as colunas relevantes do seu DataFrame
cols = df_suic_notears.columns[df_suic_notears.columns != 'Suicidio']  # Suicidio é o alvo
X = df_suic[cols]
y = df_suic['Suicidio']

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Selecionar o modelo adequado
model = RandomForestClassifier()  # ou LogisticRegression()
model.fit(X_train, y_train)

RandomForestClassifier()

In [101]:
result_tupl = recourse(
                scm, obs=obs, features=features, costs=custom_cost_function, r_type='subpopulation', t_type='counterfactual',
                predict_log_proba=model.predict_log_proba, gamma= 0.75,
                 multi_objective=False, y_name='Suicidio',
                thresh=0.6, X=df_test,
                POP_SIZE=400, NGEN=800, lbd=5000
            )

result_tupl = recourse(
                scm, obs=obs, features=features, costs=default_costs, r_type='subpopulation', t_type='counterfactual',
                predict_log_proba=model.predict_log_proba, gamma=gamma,
                multi_objective=False, y_name='Suicidio',
                thresh=thresh, X=df_test.drop(['Suicidio'], axis=1),
                POP_SIZE=400, NGEN=800, lbd=5000
            )

TypeError: unsupported operand type(s) for *: 'float' and 'function'

In [126]:
import numpy as np

# Dentro do loop de otimização ou antes de calcular o custo, faça o clip:
counterfactual = np.clip(result_tupl, bounds[feature][0], bounds[feature][1])


KeyError: 0.0

In [86]:
result_tupl
#([0.0, 0.0, 0.0,  0.0,  0.52,  0.0,  0.0, 0.0, -0.55, 0.0, -0.52], 1.0, 1.59)
#([0.0, 0.0, 0.0,  0.0,  0.51,  0.0,  0.0, 0.0, -0.51, 0.0, -0.56], 1.0, 1.58) 
#([0.0, 0.0, 0.0,  1.6,  0.0,   0.0,  0.0, 0.0, -1.51, 0.0,  0.0], 1.0, 3.1100000000000003)
#([0.0, 0.0, 0.0, -0.5,  0.53, -0.51, 0.0, 0.0,  0.0,  0.0,  0.0], 1.0, 1.54)
#([1.0, 2.0, 0.0,  1.0,  2.0,   4.0,  1.0, 1.0,  3.0,  0.0,  3.0])

([0.0, 0.0, 0.0, -0.53, 0.51, -0.53, 0.0, 0.0, 0.0, 0.0, 0.0], 1.0, 1.57)

In [99]:
for idx, feature in enumerate(lista):
        lower, upper = bounds[feature]
        lista[idx] = min(max(lista[idx], lower), upper)

KeyError: 0.0

In [96]:
lista = [0.0, 0.0, 0.0, -0.5,  0.53, -0.51, 0.0, 0.0,  0.0,  0.0]


In [89]:
feature_ranges_map = {
    'Suic_familia': [0, 1],
    'Capaz de tomar decisões importantes': [1, 2, 3, 4, 5],
    'Estudante': [0, 1],
    'Hipocondriase': [0, 1, 2, 3, 4],
    'Sentimentos_culpa': [0, 1, 2, 3, 4],
    'Trabalho e interesses': [0, 1, 2, 3, 4],
    'Dep_familia': [0, 1],
    'Alc_familia': [0, 1],
    'Capaz de desfrutar das coisas': [1, 2, 3, 4, 5],
    'Droga': [0, 1, 2]
}

In [50]:
winner, _, _ = result_tupl  # Pegando apenas o contrafactual
winner_df = pd.DataFrame([winner], columns=features)  # Converter para DataFrame

            # Predição do target contrafactual
suicidio_cf = model.predict(winner_df)[0]

In [58]:
suicidio_cf

0.0

In [78]:
df_test = df_suic_notears.drop(['Suicidio'], axis=1)[:5]
for ii in tqdm(range(len(df_test))):
        obs = df_test.iloc[ii, :]

        # Garantir que os índices de obs são strings
        obs.index = obs.index.astype(str)


100%|██████████| 5/5 [00:00<00:00, 2518.50it/s]


In [79]:
obs

Suic_familia                           1.0
Capaz de tomar decisões importantes    2.0
Estudante                              0.0
Hipocondriase                          1.0
Sentimentos_culpa                      2.0
Trabalho e interesses                  4.0
Dep_familia                            1.0
Alc_familia                            1.0
Capaz de desfrutar das coisas          3.0
Droga                                  0.0
Ansiedade                              3.0
Name: 4, dtype: float64

In [ ]:
 [0.5, 0.75, 0.85, 0.90, 0.95]

In [133]:
features = [
    "Suic_familia",
    "Capaz de tomar decisões importantes",
    "Estudante",
    "Hipocondriase",
    "Sentimentos_culpa",
    "Trabalho e interesses",
    "Dep_familia",
    "Alc_familia",
    "Capaz de desfrutar das coisas",
    "Droga",
   # "Ansiedade",
]
num_features = len(features)
default_costs = [1] * num_features  # Ou ajuste conforme necessário
# Definir as features e os limites como antes
bounds = {
    "Suic_familia": (0.0, 1.0),
    "Capaz de tomar decisões importantes": (1.0, 5.0),
    "Estudante": (0.0, 1.0),
    "Hipocondriase": (0.0, 4.0),
    "Sentimentos_culpa": (0.0, 4.0),
    "Trabalho e interesses": (0.0, 4.0),
    "Dep_familia": (0.0, 1.0),
    "Alc_familia": (0.0, 1.0),
    "Capaz de desfrutar das coisas": (1.0, 5.0),
    "Droga": (0.0, 2.0),
  #  "Ansiedade": (0.0, 4.0),
    "Suicidio": (0.0, 4.0)
}

scm.bounds = bounds
#df_test = df_suic_notears.drop(['Suicidio'], axis=1)[:5]
df_test = df_suic_notears[:5]

# Definir as features relevantes

# 3. Gerar valores dentro dos limites
def bounded_generate(context, var, bounds):
    if var not in bounds:
        print(f"Variável '{var}' não está definida em bounds. Pulando...")
        return None

    value = generator_functions[var](context)
    lower, upper = bounds[var]
    return max(min(float(value), float(upper)), float(lower))

# Gerar o contexto com base nos limites
context = {}
for var in scm.dag.var_names:
    if var in bounds:
        context[var] = bounded_generate(context, var, scm.bounds)

print("Contexto gerado para recourse:", context)

# Definir o DataFrame final com as colunas desejadas
final_df = pd.DataFrame(columns=[
    *features,                       # Colunas das features originais
    'Suicidio_Original',             # Valor original do target
    *[f'{feat}_cf' for feat in features],  # Features contrafactuais
    'Suicidio_Contrafactual'         # Valor predito com as features contrafactuais
])

# Parâmetros de recourse
thresh = 0.5
for gamma in [0.5, 0.75, 0.85, 0.90, 0.95]:
    for ii in tqdm(range(len(df_test.drop(['Suicidio'], axis=1)))):
        obs = df_test.drop(['Suicidio'], axis=1).iloc[ii, :]

        # Garantir que os índices de obs são strings
        obs.index = obs.index.astype(str)
        
        try:
            # Executar recourse
            result_tupl = recourse(
                scm, obs=obs, features=features, costs=default_costs, r_type='subpopulation', t_type='counterfactual',
                predict_log_proba=model.predict_log_proba, gamma=0.75,
                multi_objective=False, y_name='Suicidio',
                thresh=thresh, X=df_test,
                POP_SIZE=400, NGEN=800, lbd=5000
            )
            
            winner, _, _ = result_tupl  # Pegando apenas o contrafactual
            winner_df = pd.DataFrame([winner], columns=features)  # Converter para DataFrame

            # Predição do target contrafactual
            suicidio_cf = model.predict(winner_df)[0]

            # Adicionar linha ao DataFrame final
            final_df.loc[len(final_df)] = [
                *obs.values,                       # Valores originais das features
                df_test['Suicidio'],               # Target original
                *winner,                           # Valores contrafactuais das features
                suicidio_cf                        # Target predito com contrafactual
            ]
            
        except KeyError as e:
            print(f"Erro durante execução do recourse: {e}")
        except Exception as e:
            print(f"Erro inesperado: {e}")

# Exibir ou salvar o DataFrame final
print(final_df.head())  # Mostra as primeiras linhas
final_df.to_csv("resultado_contrafactual.csv", index=False)  # Salva em CSV (opcional)


Contexto gerado para recourse: {'Suic_familia': 0.0, 'Capaz de tomar decisões importantes': 1.0, 'Estudante': 0.0, 'Hipocondriase': 0.0, 'Sentimentos_culpa': 0.0, 'Trabalho e interesses': 0.0, 'Dep_familia': 0.0, 'Alc_familia': 0.0, 'Capaz de desfrutar das coisas': 1.0, 'Droga': 0.0, 'Suicidio': 0.0}



  0%|          | 0/5 [00:00<?, ?it/s]C:\Users\Cristiane\Documents\UFMG\Arquivos_Dissertacao\Projeto\icr\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Cristiane\Documents\UFMG\Arquivos_Dissertacao\Projeto\icr\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
DEBUG:icr.recourse.evaluation:CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)
DEBUG:icr.recourse.evaluation:CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)

 20%|██        | 1/5 [00:47<03:11, 47.98s/it]DEBUG:icr.recourse.evaluation:CacheInfo(hits=0, misses=0, maxsize=None, cu

KeyboardInterrupt: 

In [ ]:
final_df

In [134]:
from deap import creator

# Verifica e remove classes anteriores se existirem
if 'FitnessMin' in creator.__dict__:
    del creator.FitnessMin
if 'Individual' in creator.__dict__:
    del creator.Individual

# Agora você pode recriar as classes
from deap import base
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)


In [144]:
features = [
    "Suic_familia",
    "Capaz de tomar decisões importantes",
    "Estudante",
    "Hipocondriase",
    "Sentimentos_culpa",
    "Trabalho e interesses",
    "Dep_familia",
    "Alc_familia",
    "Capaz de desfrutar das coisas",
    "Droga",
   # "Ansiedade",
]
num_features = len(features)
default_costs = [1] * num_features  # Ou ajuste conforme necessário
# Definir as features e os limites como antes
bounds = {
    "Suic_familia": (0.0, 1.0),
    "Capaz de tomar decisões importantes": (1.0, 5.0),
    "Estudante": (0.0, 1.0),
    "Hipocondriase": (0.0, 4.0),
    "Sentimentos_culpa": (0.0, 4.0),
    "Trabalho e interesses": (0.0, 4.0),
    "Dep_familia": (0.0, 1.0),
    "Alc_familia": (0.0, 1.0),
    "Capaz de desfrutar das coisas": (1.0, 5.0),
    "Droga": (0.0, 2.0),
  #  "Ansiedade": (0.0, 4.0),
    "Suicidio": (0.0, 4.0)
}

scm.bounds = bounds
#df_test = df_suic_notears.drop(['Suicidio'], axis=1)[:5]
df_test = df_suic_notears

# Definir as features relevantes

# 3. Gerar valores dentro dos limites
def bounded_generate(context, var, bounds):
    if var not in bounds:
        print(f"Variável '{var}' não está definida em bounds. Pulando...")
        return None

    value = generator_functions[var](context)
    lower, upper = bounds[var]
    return max(min(float(value), float(upper)), float(lower))

# Gerar o contexto com base nos limites
context = {}
for var in scm.dag.var_names:
    if var in bounds:
        context[var] = bounded_generate(context, var, scm.bounds)

print("Contexto gerado para recourse:", context)

# Definir o DataFrame final com as colunas desejadas
final_df = pd.DataFrame(columns=[
    *features,                       # Colunas das features originais
    'Suicidio_Original',             # Valor original do target
    *[f'{feat}_cf' for feat in features],  # Features contrafactuais
    'Suicidio_Contrafactual'         # Valor predito com as features contrafactuais
])

# Parâmetros de recourse
thresh = 0.6
#for gamma in [0.5, 0.75, 0.85, 0.90, 0.95]:
for ii in tqdm(range(len(df_test.drop(['Suicidio'], axis=1)))):
        obs = df_test.drop(['Suicidio'], axis=1).iloc[ii, :]

        # Garantir que os índices de obs são strings
        obs.index = obs.index.astype(str)
        
        try:
            # Executar recourse
            result_tupl = recourse(
                scm, obs=obs, features=features, costs=default_costs, r_type='subpopulation', t_type='counterfactual',
                predict_log_proba=model.predict_log_proba, gamma=0.75,
                multi_objective=False, y_name='Suicidio',
                thresh=thresh, X=df_test.drop(['Suicidio'], axis=1),
                POP_SIZE=400, NGEN=800, lbd=5000
            )
            
            winner, _, _ = result_tupl  # Pegando apenas o contrafactual
            winner_df = pd.DataFrame([winner], columns=features)  # Converter para DataFrame

            # Predição do target contrafactual
            suicidio_cf = model.predict(winner_df)[0]

            # Adicionar linha ao DataFrame final
            final_df.loc[len(final_df)] = [
                *obs.values,                       # Valores originais das features
                df_test.loc[ii, 'Suicidio'],               # Target original
                *winner,                           # Valores contrafactuais das features
                suicidio_cf                        # Target predito com contrafactual
            ]
            
        except KeyError as e:
            print(f"Erro durante execução do recourse: {e}")
        except Exception as e:
            print(f"Erro inesperado: {e}")

# Exibir ou salvar o DataFrame final
#print(final_df.head())  # Mostra as primeiras linhas
final_df.to_csv("resultado_contrafactual.csv", index=False)  # Salva em CSV (opcional)

Contexto gerado para recourse: {'Suic_familia': 0.0, 'Capaz de tomar decisões importantes': 1.0, 'Estudante': 0.0, 'Hipocondriase': 0.0, 'Sentimentos_culpa': 0.0, 'Trabalho e interesses': 0.0, 'Dep_familia': 0.0, 'Alc_familia': 0.0, 'Capaz de desfrutar das coisas': 1.0, 'Droga': 0.0, 'Suicidio': 0.0}



  0%|          | 0/3953 [00:00<?, ?it/s]DEBUG:icr.recourse.evaluation:CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)
DEBUG:icr.recourse.evaluation:CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)

  0%|          | 1/3953 [00:45<49:43:16, 45.29s/it]DEBUG:icr.recourse.evaluation:CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)
DEBUG:icr.recourse.evaluation:CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)

  0%|          | 2/3953 [01:29<48:44:35, 44.41s/it]DEBUG:icr.recourse.evaluation:CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)
DEBUG:icr.recourse.evaluation:CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)

  0%|          | 3/3953 [02:15<49:37:29, 45.23s/it]DEBUG:icr.recourse.evaluation:CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)
DEBUG:icr.recourse.evaluation:CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)

  0%|          | 4/3953 [02:58<48:47:29, 44.48s/it]DEBUG:icr.recourse.evaluation:CacheInfo(hits=0, misses=0, maxsize=None, currsiz

In [139]:
df_test.loc[1, 'Suicidio']

1.0

In [137]:
df_test['Suicidio']

0    2.0
1    1.0
Name: Suicidio, dtype: float64

In [146]:
final_df.Suicidio_Contrafactual.value_counts()

 1.0    1895
 2.0    1761
-1.0     294
 0.0       3
Name: Suicidio_Contrafactual, dtype: int64

In [155]:
final_df['Id'] = final_df.index

In [156]:
final_df2 = final_df.copy()

In [157]:
#colunas_para_filtrar = ['feature1', 'feature2', 'target']  # Substitua pelas colunas reais
final_df2 = final_df2[(final_df2 >= 0).all(axis=1)]


In [158]:
final_df2

,Suic_familia,Capaz de tomar decisões importantes,Estudante,Hipocondriase,Sentimentos_culpa,Trabalho e interesses,Dep_familia,Alc_familia,Capaz de desfrutar das coisas,Droga,...,Estudante_cf,Hipocondriase_cf,Sentimentos_culpa_cf,Trabalho e interesses_cf,Dep_familia_cf,Alc_familia_cf,Capaz de desfrutar das coisas_cf,Droga_cf,Suicidio_Contrafactual,Id
2,1.0,2.0,0.0,0.0,3.0,3.0,1.0,1.0,2.0,0.0,...,0.00,0.00,0.0,0.00,0.0,0.0,0.53,0.0,1.0,2
4,1.0,2.0,0.0,1.0,2.0,4.0,1.0,1.0,3.0,0.0,...,0.54,0.00,0.0,0.00,0.0,0.0,0.00,0.0,1.0,4
5,1.0,2.0,0.0,0.0,3.0,3.0,1.0,1.0,2.0,0.0,...,0.00,0.00,0.0,0.00,0.0,0.0,0.57,0.0,1.0,5
6,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,...,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,2.0,6
7,1.0,2.0,0.0,0.0,2.0,3.0,1.0,1.0,2.0,0.0,...,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,1.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3943,1.0,2.0,0.0,1.0,2.0,3.0,1.0,1.0,2.0,0.0,...,0.00,0.53,0.0,0.00,0.0,0.0,0.00,0.0,1.0,3943
3944,1.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,0.0,...,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,2.0,3944
3945,1.0,2.0,0.0,3.0,0.0,3.0,1.0,1.0,1.0,0.0,...,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,2.0,3945
3951,1.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,...,0.00,0.58,0.0,0.00,0.0,0.0,0.00,0.0,2.0,3951


In [159]:
import numpy as np
final_df2 = np.ceil(final_df2)

In [160]:
final_df2.to_csv("icr.csv", index=False)  # Salva em CSV (opcional)

In [21]:
'''num_features = len(features)
default_costs = [1] * num_features  # Ou ajuste conforme necessário
# Definir as features e os limites como antes
bounds = {
    "Suic_familia": (0.0, 1.0),
    "Capaz de tomar decisões importantes": (1.0, 5.0),
    "Estudante": (0.0, 1.0),
    "Hipocondriase": (0.0, 4.0),
    "Sentimentos_culpa": (0.0, 4.0),
    "Trabalho e interesses": (0.0, 4.0),
    "Dep_familia": (0.0, 1.0),
    "Alc_familia": (0.0, 1.0),
    "Capaz de desfrutar das coisas": (1.0, 5.0),
    "Droga": (0.0, 2.0),
    "Ansiedade": (0.0, 4.0),
    "Suicidio": (0.0, 4.0)
}

scm.bounds = bounds

# Definir as features relevantes
features = [
    "Suic_familia",
    "Capaz de tomar decisões importantes",
    "Estudante",
    "Hipocondriase",
    "Sentimentos_culpa",
    "Trabalho e interesses",
    "Dep_familia",
    "Alc_familia",
    "Capaz de desfrutar das coisas",
    "Droga",
    "Ansiedade",
]
# 3. Gerar valores dentro dos limites
def bounded_generate(context, var, bounds):
    if var not in bounds:
        print(f"Variável '{var}' não está definida em bounds. Pulando...")
        return None

    value = generator_functions[var](context)
    lower, upper = bounds[var]
    return max(min(float(value), float(upper)), float(lower))

# Gerar o contexto com base nos limites
context = {}
for var in scm.dag.var_names:
    if var in bounds:
        context[var] = bounded_generate(context, var, scm.bounds)

print("Contexto gerado para recourse:", context)

# Definir parâmetros para recourse
thresh = 0.5
for gamma in [0.5, 0.75, 0.85, 0.90, 0.95]:
    for ii in tqdm(range(len(X_test))):
        obs = X_test.iloc[ii, :]

        # Garantir que os índices de obs são strings
        obs.index = obs.index.astype(str)
        
        # Validar alinhamento entre obs e features
        if not set(features).issubset(obs.index):
            print(f"Features faltando em obs: {set(features) - set(obs.index)}")
            continue

        # Executar recourse
        try:
            result_tupl = recourse(
                scm, obs=obs, features=features, costs=default_costs, r_type='subpopulation', t_type='counterfactual',
                predict_log_proba=model.predict_log_proba, gamma=gamma,
                multi_objective=False, y_name='Suicidio',
                thresh=thresh, X=X_test,
                POP_SIZE=400, NGEN=800, lbd=5000
            )
            winner, goal_cost, intv_cost = result_tupl
            print(f'Gamma: {gamma}, Winner: {winner}, Goal cost: {goal_cost}, Intervention cost: {intv_cost}')
        except KeyError as e:
            print(f"Erro durante execução do recourse: {e}")

# Testar o modelo preditivo separadamente
test_obs = X_test.iloc[0]  # Exemplo de observação
print(model.predict_log_proba([test_obs]))'''

Contexto gerado para recourse: {'Suic_familia': 0.0, 'Capaz de tomar decisões importantes': 1.0, 'Estudante': 0.0, 'Hipocondriase': 0.0, 'Sentimentos_culpa': 0.0, 'Trabalho e interesses': 0.0, 'Dep_familia': 0.0, 'Alc_familia': 0.0, 'Capaz de desfrutar das coisas': 1.0, 'Droga': 0.0, 'Ansiedade': 0.0, 'Suicidio': 0.0}


  0%|          | 0/1186 [00:00<?, ?it/s]C:\Users\Cristiane\Documents\UFMG\Arquivos_Dissertacao\Projeto\icr\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Cristiane\Documents\UFMG\Arquivos_Dissertacao\Projeto\icr\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
  0%|          | 0/1186 [10:21<?, ?it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (400,) + inhomogeneous part.

In [323]:
# Definir os limites para cada variável (ajuste conforme necessário)
'''bounds = {
    "Suic_familia": (0, 1),
    "Capaz de tomar decisões importantes": (1, 5),
    "Estudante": (0, 1),
    "Hipocondriase": (0, 4),
    "Sentimentos_culpa": (0, 4),
    "Trabalho e interesses": (0, 4),
    "Dep_familia": (0, 1),
    "Alc_familia": (0, 1),
    "Capaz de desfrutar das coisas": (1, 5),
    "Droga": (0, 2),
    "Ansiedade": (0, 4)   
}

features = [
   "Suic_familia",
    "Capaz de tomar decisões importantes",
    "Estudante",
    "Hipocondriase",
    "Sentimentos_culpa",
    "Trabalho e interesses",
    'Dep_familia',
    'Alc_familia',
    'Capaz de desfrutar das coisas',
    'Droga',    
    'Ansiedade'
     ]

# Adicionar os limites ao SCM
bounds = []
valid_features = [feature for feature in features if feature in scm.bounds]

obs = obs.rename_axis(None).rename(columns=str)

for node in valid_features:
    try:
        scm_bounds = scm.bounds[node]
        tupl = (scm_bounds[0] - obs[node], scm_bounds[1] - obs[node])
    except KeyError as e:
        print(f"Erro ao acessar bounds para {node}: {e}")

def bounded_generate(context, var, bounds):
    value = generator_functions[var](context)
    lower = bounds[var][0]
    upper = bounds[var][1]

    print(f"Antes de ajuste: value={value}, lower={lower}, upper={upper}")
    
    if isinstance(value, pd.Series):
        value = value.iloc[0]
    if isinstance(lower, pd.Series):
        lower = lower.iloc[0]
    if isinstance(upper, pd.Series):
        upper = upper.iloc[0]

    print(f"Depois de ajuste: value={value}, lower={lower}, upper={upper}")
    
    return max(min(float(value), float(upper)), float(lower))

# Atualizar o contexto usando os limites
for var in scm.dag.var_names:
    if var not in context:
        if var in generator_functions:
            context[var] = bounded_generate(context, var, scm.bounds)
# Executar recourse após adicionar limites
thresh = 0.5
for gamma in [0.5, 0.75, 0.85, 0.90, 0.95]:
    for ii in tqdm(range(len(X_test))):
        obs = X_test.iloc[ii, :]
        
        # Verificar consistência de índices/colunas
        if not set(valid_features).issubset(obs.index):
            print(f"Features faltando em obs: {set(valid_features) - set(obs.index)}")
            continue
        
        # Executar recourse
        try:
            result_tupl = recourse(
                scm, obs, None, 'subpopulation', 'improvement',
                predict_log_proba=model.predict_log_proba, gamma=gamma,
                multi_objective=False, y_name='Suicidio',
                thresh=thresh, X=X_test,
                POP_SIZE=400, NGEN=800, lbd=5000, t_type='counterfactual'
            )
            winner, goal_cost, intv_cost = result_tupl
            print(f'Gamma: {gamma}, Winner: {winner}, Goal cost: {goal_cost}, Intervention cost: {intv_cost}')
        except KeyError as e:
            print(f"Erro durante execução do recourse: {e}")'''


'bounds = {\n    "Suic_familia": (0, 1),\n    "Capaz de tomar decisões importantes": (1, 5),\n    "Estudante": (0, 1),\n    "Hipocondriase": (0, 4),\n    "Sentimentos_culpa": (0, 4),\n    "Trabalho e interesses": (0, 4),\n    "Dep_familia": (0, 1),\n    "Alc_familia": (0, 1),\n    "Capaz de desfrutar das coisas": (1, 5),\n    "Droga": (0, 2),\n    "Ansiedade": (0, 4)   \n}\n\nfeatures = [\n   "Suic_familia",\n    "Capaz de tomar decisões importantes",\n    "Estudante",\n    "Hipocondriase",\n    "Sentimentos_culpa",\n    "Trabalho e interesses",\n    \'Dep_familia\',\n    \'Alc_familia\',\n    \'Capaz de desfrutar das coisas\',\n    \'Droga\',    \n    \'Ansiedade\'\n     ]\n\n# Adicionar os limites ao SCM\nbounds = []\nvalid_features = [feature for feature in features if feature in scm.bounds]\n\nfor node in valid_features:\n    try:\n        scm_bounds = scm.bounds[node]\n        tupl = (scm_bounds[0] - obs[node], scm_bounds[1] - obs[node])\n    except KeyError as e:\n        print(f

In [427]:
'''from tqdm import tqdm

# Definir os limites para cada variável (ajuste conforme necessário)
bounds = {
    "Suic_familia": (0.0, 1.0),
    "Capaz de tomar decisões importantes": (1.0, 5.0),
    "Estudante": (0.0, 1.0),
    "Hipocondriase": (0.0, 4.0),
    "Sentimentos_culpa": (0.0, 4.0),
    "Trabalho e interesses": (0.0, 4.0),
    "Dep_familia": (0.0, 1.0),
    "Alc_familia": (0.0, 1.0),
    "Capaz de desfrutar das coisas": (1.0, 5.0),
    "Droga": (0.0, 2.0),
    "Ansiedade": (0.0, 4.0),
    "Suicidio": (0.0, 4.0)
}

scm.bounds = bounds
# Definir as features relevantes
features = [
    "Suic_familia",
    "Capaz de tomar decisões importantes",
    "Estudante",
    "Hipocondriase",
    "Sentimentos_culpa",
    "Trabalho e interesses",
    "Dep_familia",
    "Alc_familia",
    "Capaz de desfrutar das coisas",
    "Droga",
    "Ansiedade",
]

# Validar se todas as features têm limites definidos
valid_features = [feature for feature in features if feature in scm.bounds]
for feature in features:
    if feature not in scm.bounds:
        print(f"A chave '{feature}' não foi encontrada em scm.bounds. Ignorando...")

# Atualizar os limites no SCM (se necessário)
for node in valid_features:
    try:
        scm_bounds = scm.bounds[node]
        print(f"Bounds para {node}: {scm_bounds}")
    except KeyError as e:
        print(f"Erro ao acessar bounds para {node}: {e}")

# Função para gerar valores dentro dos limites
def bounded_generate(context, var, bounds):
    if var not in bounds:
        print(f"Variável '{var}' não está definida em bounds. Pulando...")
        return None

    value = generator_functions[var](context)
    lower = bounds[var][0]
    upper = bounds[var][1]

    print(f"Antes de ajuste: value={value}, lower={lower}, upper={upper}")
    
    if isinstance(value, pd.Series):
        value = value.iloc[0]
    if isinstance(lower, pd.Series):
        lower = lower.iloc[0]
    if isinstance(upper, pd.Series):
        upper = upper.iloc[0]

    print(f"Depois de ajuste: value={value}, lower={lower}, upper={upper}")
    
    return max(min(float(value), float(upper)), float(lower))

# Se scm.dag.var_names for um numpy.ndarray, converta para lista
var_names_list = list(scm.dag.var_names)

# Remover 'Suicidio' se presente
if 'Suicidio' in var_names_list:
    var_names_list.remove('Suicidio')

# Agora você pode fazer algo com a lista var_names_list
# Filtrando diretamente se for um numpy.ndarray
#scm.dag.var_names = scm.dag.var_names[scm.dag.var_names != 'Suicidio']
# Verifique se scm.dag.var_names é um ndarray ou uma lista
if isinstance(scm.dag.var_names, np.ndarray):
    scm.dag.var_names = scm.dag.var_names.tolist()  # Converte para lista se necessário

# Remover 'Suicidio' da lista
if 'Suicidio' in scm.dag.var_names:
    scm.dag.var_names.remove('Suicidio')
    
bounds["Suicidio"] = (0, 4)  # Ajuste conforme necessário.

# Atualizar o contexto usando os limites
context = {}
for var in scm.dag.var_names:
    if var in generator_functions:
        if var not in bounds:
            print(f"Bounds não definidos para '{var}'. Pulando...")
            continue
        context[var] = bounded_generate(context, var, scm.bounds)
# Mostrar o contexto gerado antes de passar para recourse
print("Contexto gerado para recourse:", context)

# Configurar parâmetros para recourse
thresh = 0.5
for gamma in [0.5, 0.75, 0.85, 0.90, 0.95]:
    for ii in tqdm(range(len(X_test))):
        obs = X_test.iloc[ii, :]

        # Garantir que os índices de obs são strings
        obs.index = obs.index.astype(str)
        
        # Validar alinhamento entre obs e features
        if not set(features).issubset(obs.index):
            print(f"Features faltando em obs: {set(features) - set(obs.index)}")
            continue

        # Executar recourse
        try:
            result_tupl = recourse(
                scm, obs=obs, features=features, costs=None,r_type='subpopulation', t_type='counterfactual',
                predict_log_proba=model.predict_log_proba, gamma=gamma,
                multi_objective=False, y_name='Suicidio',
                thresh=thresh, X=X_test,
                POP_SIZE=400, NGEN=800, lbd=5000
            )
            winner, goal_cost, intv_cost = result_tupl
            print(f'Gamma: {gamma}, Winner: {winner}, Goal cost: {goal_cost}, Intervention cost: {intv_cost}')
        except KeyError as e:
            print(f"Erro durante execução do recourse: {e}")
            
# Testar o modelo preditivo separadamente
test_obs = X_test.iloc[0]  # Exemplo de observação
print(model.predict_log_proba([test_obs]))'''

Bounds para Suic_familia: (0.0, 1.0)
Bounds para Capaz de tomar decisões importantes: (1.0, 5.0)
Bounds para Estudante: (0.0, 1.0)
Bounds para Hipocondriase: (0.0, 4.0)
Bounds para Sentimentos_culpa: (0.0, 4.0)
Bounds para Trabalho e interesses: (0.0, 4.0)
Bounds para Dep_familia: (0.0, 1.0)
Bounds para Alc_familia: (0.0, 1.0)
Bounds para Capaz de desfrutar das coisas: (1.0, 5.0)
Bounds para Droga: (0.0, 2.0)
Bounds para Ansiedade: (0.0, 4.0)
Antes de ajuste: value=0, lower=0.0, upper=1.0
Depois de ajuste: value=0, lower=0.0, upper=1.0
Antes de ajuste: value=1, lower=1.0, upper=5.0
Depois de ajuste: value=1, lower=1.0, upper=5.0
Antes de ajuste: value=0, lower=0.0, upper=1.0
Depois de ajuste: value=0, lower=0.0, upper=1.0
Antes de ajuste: value=0, lower=0.0, upper=4.0
Depois de ajuste: value=0, lower=0.0, upper=4.0
Antes de ajuste: value=0, lower=0.0, upper=4.0
Depois de ajuste: value=0, lower=0.0, upper=4.0
Antes de ajuste: value=0, lower=0.0, upper=4.0
Depois de ajuste: value=0, lowe

  0%|          | 0/1186 [00:00<?, ?it/s]C:\Users\Cristiane\Documents\UFMG\Arquivos_Dissertacao\Projeto\icr\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Cristiane\Documents\UFMG\Arquivos_Dissertacao\Projeto\icr\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
  0%|          | 0/1186 [00:04<?, ?it/s]


NotImplementedError: only BinomialBinary or GenericSCM supported.

In [365]:
'''from imblearn.over_sampling import SMOTE
from tqdm import tqdm
import numpy as np

# Durante o treinamento e na análise do recourse, remova 'Suicidio' de X
X_train = X_train.drop(columns=['Suicidio'], errors='ignore')
X_test = X_test.drop(columns=['Suicidio'], errors='ignore')

# 1. Aplicação do SMOTE para classes minoritárias
target_counts = {3.0: 500, 4.0: 500}
X_resample, y_resample = X_train.copy(), y_train.copy()

for class_label, desired_count in target_counts.items():
    current_count = y_resample[y_resample == class_label].shape[0]
    if current_count < desired_count:
        # Definir k_neighbors menor que o número atual de amostras
        k_neighbors = min(5, current_count - 1) if current_count > 1 else 1
        smote = SMOTE(sampling_strategy={class_label: desired_count}, k_neighbors=k_neighbors)
        X_resample, y_resample = smote.fit_resample(X_resample, y_resample)
        print(f"SMOTE aplicado com sucesso para classe {class_label} com {k_neighbors} vizinhos.")

# Remover a classe -1.0 se existir
mask = y_resample != -1.0
X_resample = X_resample[mask]
y_resample = y_resample[mask]
print("Distribuição das classes após SMOTE e filtragem:", y_resample.value_counts())

# 2. Definição dos limites para cada variável
bounds = {
    "Suic_familia": (0, 1),
    "Capaz de tomar decisões importantes": (1, 5),
    "Estudante": (0, 1),
    "Hipocondriase": (0, 4),
    "Sentimentos_culpa": (0, 4),
    "Trabalho e interesses": (0, 4),
    "Dep_familia": (0, 1),
    "Alc_familia": (0, 1),
    "Capaz de desfrutar das coisas": (1, 5),
    "Droga": (0, 2),
    "Ansiedade": (0, 4),
    "Suicidio": (0, 4)  # Adicionado aqui também
}

# 3. Gerar valores dentro dos limites
def bounded_generate(context, var, bounds):
    if var not in bounds:
        print(f"Variável '{var}' não está definida em bounds. Pulando...")
        return None

    value = generator_functions[var](context)
    lower, upper = bounds[var]
    return max(min(float(value), float(upper)), float(lower))

# 4. Remover 'Suicidio' da lista de variáveis SCM
var_names_list = list(scm.dag.var_names)
if 'Suicidio' in var_names_list:
    var_names_list.remove('Suicidio')

# Atualizar o contexto usando os limites
context = {}
for var in var_names_list:
    if var in generator_functions:
        context[var] = bounded_generate(context, var, bounds)

# 5. Loop para execução do recourse
thresh = 0.5
for gamma in [0.5, 0.75, 0.85, 0.90, 0.95]:
    for ii in tqdm(range(len(X_test))):
        obs = X_test.iloc[ii, :].copy()
        obs.index = obs.index.astype(str)

        # Garantir que todas as features estão presentes em obs
        # Remover 'Suicidio' de bounds, pois é o target e não uma feature
        bounds_filtered = {k: v for k, v in bounds.items() if k != 'Suicidio'}
        
        # Verificar se as features de entrada estão presentes em obs
        missing_features = set(bounds_filtered.keys()) - set(obs.index)  # Comparar com as features de entrada
        
        if missing_features:
            print(f"Features faltando: {missing_features}")
            continue
        
        # Executar o recourse
        try:
            result_tupl = recourse(
                scm, obs, None, 'subpopulation', 'improvement',
                predict_log_proba=model.predict_log_proba, gamma=gamma,
                multi_objective=False, y_name='Suicidio',
                thresh=thresh, X=X_test,
                POP_SIZE=400, NGEN=800, lbd=5000, t_type='counterfactual'
            )
            winner, goal_cost, intv_cost = result_tupl
            print(f'Gamma: {gamma}, Winner: {winner}, Goal cost: {goal_cost}, Intervention cost: {intv_cost}')
        except KeyError as e:
            print(f"Erro durante execução do recourse: {e}")

        # Adicionar o ajuste de clipping para probabilidade (para evitar inf)
        probas = model.predict_proba([obs])
        probas_clipped = np.clip(probas, 1e-6, 1 - 1e-6)  # Ajuste de clipping para evitar valores extremos
        print("Probabilidades ajustadas:", probas_clipped)

# Teste final do modelo preditivo
test_obs = X_test.iloc[0]
print(model.predict_log_proba([test_obs]))'''

'from imblearn.over_sampling import SMOTE\nfrom tqdm import tqdm\nimport numpy as np\n\n# Durante o treinamento e na análise do recourse, remova \'Suicidio\' de X\nX_train = X_train.drop(columns=[\'Suicidio\'], errors=\'ignore\')\nX_test = X_test.drop(columns=[\'Suicidio\'], errors=\'ignore\')\n\n# 1. Aplicação do SMOTE para classes minoritárias\ntarget_counts = {3.0: 500, 4.0: 500}\nX_resample, y_resample = X_train.copy(), y_train.copy()\n\nfor class_label, desired_count in target_counts.items():\n    current_count = y_resample[y_resample == class_label].shape[0]\n    if current_count < desired_count:\n        # Definir k_neighbors menor que o número atual de amostras\n        k_neighbors = min(5, current_count - 1) if current_count > 1 else 1\n        smote = SMOTE(sampling_strategy={class_label: desired_count}, k_neighbors=k_neighbors)\n        X_resample, y_resample = smote.fit_resample(X_resample, y_resample)\n        print(f"SMOTE aplicado com sucesso para classe {class_label} co

In [407]:
'''from imblearn.over_sampling import SMOTE
from tqdm import tqdm
import numpy as np
# Durante o treinamento e na análise do recourse, remova 'Suicidio' de X
X_train = X_train.drop(columns=['Suicidio'], errors='ignore')
X_test = X_test.drop(columns=['Suicidio'], errors='ignore')

# 1. Aplicação do SMOTE para classes minoritárias
target_counts = {3.0: 500, 4.0: 500}
X_resample, y_resample = X_train.copy(), y_train.copy()

for class_label, desired_count in target_counts.items():
    current_count = y_resample[y_resample == class_label].shape[0]
    if current_count < desired_count:
        # Definir k_neighbors menor que o número atual de amostras
        k_neighbors = min(5, current_count - 1) if current_count > 1 else 1
        smote = SMOTE(sampling_strategy={class_label: desired_count}, k_neighbors=k_neighbors)
        X_resample, y_resample = smote.fit_resample(X_resample, y_resample)
        print(f"SMOTE aplicado com sucesso para classe {class_label} com {k_neighbors} vizinhos.")

# Remover a classe -1.0 se existir
mask = y_resample != -1.0
X_resample = X_resample[mask]
y_resample = y_resample[mask]
print("Distribuição das classes após SMOTE e filtragem:", y_resample.value_counts())

# 2. Definição dos limites para cada variável
bounds = {
    "Suic_familia": (0.0, 1.0),
    "Capaz de tomar decisões importantes": (1.0, 5.0),
    "Estudante": (0.0, 1.0),
    "Hipocondriase": (0.0, 4.0),
    "Sentimentos_culpa": (0.0, 4.0),
    "Trabalho e interesses": (0.0, 4.0),
    "Dep_familia": (0.0, 1.0),
    "Alc_familia": (0.0, 1.0),
    "Capaz de desfrutar das coisas": (1.0, 5.0),
    "Droga": (0.0, 2.0),
    "Ansiedade": (0.0, 4.0),
    "Suicidio": (0.0, 4.0)  # Adicionado aqui também
}
features = [
   "Suic_familia",
    "Capaz de tomar decisões importantes",
    "Estudante",
    "Hipocondriase",
    "Sentimentos_culpa",
    "Trabalho e interesses",
    'Dep_familia',
    'Alc_familia',
    'Capaz de desfrutar das coisas',
    'Droga',    
    'Ansiedade'
     ]

# Adicionar os limites ao SCM
bounds = []
valid_features = [feature for feature in features if feature in scm.bounds]

for node in valid_features:
    try:
        scm_bounds = scm.bounds[node]       
    except KeyError as e:
        print(f"Erro ao acessar bounds para {node}: {e}")


# 3. Gerar valores dentro dos limites
def bounded_generate(context, var, bounds):
    if var not in bounds:
        print(f"Variável '{var}' não está definida em bounds. Pulando...")
        return None

    value = generator_functions[var](context)
    lower, upper = bounds[var]
    return max(min(float(value), float(upper)), float(lower))

# 4. Remover 'Suicidio' da lista de variáveis SCM
var_names_list = list(scm.dag.var_names)
if 'Suicidio' in var_names_list:
    var_names_list.remove('Suicidio')

# Atualizar o contexto usando os limites
context = {}
for var in var_names_list:
    if var in generator_functions:
        context[var] = bounded_generate(context, var, bounds)

# 5. Loop para execução do recourse
thresh = 0.5
for gamma in [0.5, 0.75, 0.85, 0.90, 0.95]:
    for ii in tqdm(range(len(X_test))):
        obs = X_test.iloc[ii, :].copy()
        obs.index = obs.index.astype(str)

        # Garantir que todas as features estão presentes em obs
        # Remover 'Suicidio' de bounds, pois é o target e não uma feature
        bounds_filtered = {k: v for k, v in bounds.items() if k != 'Suicidio'}
        
        # Verificar se as features de entrada estão presentes em obs
        missing_features = set(bounds_filtered.keys()) - set(obs.index)  # Comparar com as features de entrada
        
        if missing_features:
            print(f"Features faltando: {missing_features}")
            continue
        
        # Executar o recourse
        try:
            result_tupl = recourse(
                scm, obs, None, 'subpopulation', 'improvement',
                predict_log_proba=model.predict_log_proba, gamma=gamma,
                multi_objective=False, y_name='Suicidio',
                thresh=thresh, X=X_test,
                POP_SIZE=400, NGEN=800, lbd=5000, t_type='counterfactual'
            )
            winner, goal_cost, intv_cost = result_tupl
            print(f'Gamma: {gamma}, Winner: {winner}, Goal cost: {goal_cost}, Intervention cost: {intv_cost}')
        except KeyError as e:
            print(f"Erro durante execução do recourse: {e}")

# Teste final do modelo preditivo
test_obs = X_test.iloc[0]
print(model.predict_log_proba([test_obs]))'''

SMOTE aplicado com sucesso para classe 3.0 com 5 vizinhos.
SMOTE aplicado com sucesso para classe 4.0 com 4 vizinhos.
Distribuição das classes após SMOTE e filtragem: 1.0    1074
0.0    1021
2.0     601
3.0     500
4.0     500
Name: Suicidio, dtype: int64
Variável 'Suic_familia' não está definida em bounds. Pulando...
Variável 'Capaz de tomar decisões importantes' não está definida em bounds. Pulando...
Variável 'Estudante' não está definida em bounds. Pulando...
Variável 'Hipocondriase' não está definida em bounds. Pulando...
Variável 'Sentimentos_culpa' não está definida em bounds. Pulando...
Variável 'Trabalho e interesses' não está definida em bounds. Pulando...
Variável 'Dep_familia' não está definida em bounds. Pulando...
Variável 'Alc_familia' não está definida em bounds. Pulando...
Variável 'Capaz de desfrutar das coisas' não está definida em bounds. Pulando...
Variável 'Droga' não está definida em bounds. Pulando...
Variável 'Ansiedade' não está definida em bounds. Pulando...


  0%|          | 0/1186 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'items'

In [ ]:
'''from icr.causality.scms.examples import scm_dict
from icr.recourse import recourse
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from icr.causality.utils import indvd_to_intrv
import pandas as pd
import argparse
from tqdm import tqdm

scms = ['3var-causal', '3var-noncausal', '5var-skill', '7var-covid']
model_dict = {'3var-causal': 'logreg', '3var-noncausal': 'logreg',
              '5var-skill': 'rf',
              '7var-covid': 'rf'}
scms = ['3var-causal']

use_scm_pred = False
lbd = 5000

N = 10000
test_size = round(0.3 * N)

gammas = [0.5, 0.75, 0.85, 0.90, 0.95]

results = {}

scm_name = scms[0]

N_INDIVID = 5
thresh = 0.5
SUBPOPULATION_SIZE = 10**3
MAX_EVAL = 20
parser = argparse.ArgumentParser("")

parser.add_argument("savepath", help="path to savepath", type=str)
parser.add_argument("--N", help="number of runs", type=int, default=N_INDIVID)

N_INDIVID = 3953

for scm_name in scms:
        print(scm_name)

        df_res = []

        scm = scm_dict[scm_name]

        context = scm.sample_context(N)
        data = scm.compute()

        cols = data.columns[data.columns != scm.predict_target]
        X = data.loc[:, cols]
        y = data.loc[:, scm.predict_target]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

        if model_dict[scm_name] == 'rf':
            model = RandomForestClassifier()
        elif model_dict[scm_name] == 'logreg':
            model = LogisticRegression()
        else:
            raise RuntimeError('model class not available')

        model.fit(X_train, y_train)

        ixs_rejected = np.exp(model.predict_log_proba(X_test)[:, 1]) <= thresh
        ixs_rejected = np.arange(len(X_test))[ixs_rejected]'''


In [294]:
'''from icr.causality.scms.generic import GenericSCM
from icr.causality import DirectedAcyclicGraph

import numpy as np
from icr.causality import DirectedAcyclicGraph

# Definir as variáveis e as dependências (dicionário de adjacências)
adj = {
    'Suic_familia': ['Suicidio'],
    'Capaz de tomar decisões importantes': ['Suicidio'],
    'Estudante': ['Suicidio'],
    'Hipocondriase': ['Suicidio'],
    'Sentimentos_culpa': ['Suicidio'],
    'Trabalho e interesses': ['Suicidio'],
    'Dep_familia': ['Suicidio'],
    'Alc_familia': ['Suicidio'],
    'Capaz de desfrutar das coisas': ['Suicidio'],
    'Droga': ['Suicidio'],
    'Ansiedade': ['Suicidio']
}

# Lista de variáveis
var_names = list(adj.keys()) + ['Suicidio']

# Criar a matriz de adjacência
adj_matrix = np.zeros((len(var_names), len(var_names)))
for parent, children in adj.items():
    for child in children:
        parent_idx = var_names.index(parent)
        child_idx = var_names.index(child)
        adj_matrix[parent_idx, child_idx] = 1

# Criar o DAG a partir da matriz de adjacência
dag = DirectedAcyclicGraph(adjacency_matrix=adj_matrix, var_names=var_names)

print("DAG criado com sucesso!")
print("Nomes das variáveis:", dag.var_names)
# Verifique o DAG
print("Matriz de Adjacência:")
#print(dag.adjacency_matrix)
from icr.causality.scms.generic import GenericSCM

# Definir funções geradoras para o SCM
def generate_suicidio(context):
    suicidio = (
        context.get("Suic_familia", 0) * 0.4 +
        context.get("Capaz de tomar decisões importantes", 0) * 0.3 +
        context.get("Ansiedade", 0) * 0.3
    )
    return round(suicidio)

# Outras funções geradoras (valores padrão como exemplo)
generator_functions = {
    "Suic_familia": lambda context: context.get("Suic_familia", 0),
    "Capaz de tomar decisões importantes": lambda context: context.get("Capaz de tomar decisões importantes", 1),
    "Estudante": lambda context: context.get("Estudante", 0),
    "Hipocondriase": lambda context: context.get("Hipocondriase", 0),
    "Sentimentos_culpa": lambda context: context.get("Sentimentos_culpa", 0),
    "Trabalho e interesses": lambda context: context.get("Trabalho e interesses", 0),
    "Dep_familia": lambda context: context.get("Dep_familia", 0),
    "Alc_familia": lambda context: context.get("Alc_familia", 0),
    "Capaz de desfrutar das coisas": lambda context: context.get("Capaz de desfrutar das coisas", 1),
    "Droga": lambda context: context.get("Droga", 0),
    "Ansiedade": lambda context: context.get("Ansiedade", 0),
    "Suicidio": generate_suicidio,
}

# Criar o SCM
scm = GenericSCM(generator_functions)

# Testar o SCM
samples = scm.sample(n_samples=5)
print(samples)'''


DAG criado com sucesso!
Nomes das variáveis: ['Suic_familia' 'Capaz de tomar decisões importantes' 'Estudante'
 'Hipocondriase' 'Sentimentos_culpa' 'Trabalho e interesses' 'Dep_familia'
 'Alc_familia' 'Capaz de desfrutar das coisas' 'Droga' 'Ansiedade'
 'Suicidio']
Matriz de Adjacência:


AttributeError: 'dict' object has no attribute 'var_names'

In [ ]:
'''

if __name__ == "__main__":
    # savepath = '../experiments/eval-acceptance-subp/'

    

    # iterate over the different scms
    

        # for each scm compute recourse recommendation(s) for N_INDIV individuals
        for gamma in gammas:
            print(f'\t{scm_name}, gamma: {gamma}')
            for ii in tqdm(range(N_INDIVID)):
                # print(f'individual: {ii}')

                ix = ixs_rejected[ii]

                obs = X_test.iloc[ix, :]

                result_tupl = recourse(scm, cols, obs, scm.costs, 'subpopulation', 'improvement',
                                       predict_log_proba=model.predict_log_proba, gamma=gamma,
                                       multi_objective=False, y_name=scm.predict_target,
                                       thresh=thresh, X=X_test,
                                       POP_SIZE=400, NGEN=800, lbd=lbd)

                winner, goal_cost, intv_cost = result_tupl
                # print(winner)

                indivduals = np.array([winner])
                values = np.array([[goal_cost]])

                # we already have the meaningfulness estimates
                # we additionally need the subpopulation-based acceptance rate
                # and the acceptance bound

                df_res_ind = []

                # iterate over individuals to get the quantities
                for jj in range(min(MAX_EVAL, len(indivduals))):

                    ind = indivduals[jj, :]
                    meaningfulness = values[jj, 0]

                    # sample the subgroub
                    scm_cp = scm.copy()
                    acs = scm_cp.dag.get_ancestors_node(scm_cp.predict_target)
                    intv_dict = indvd_to_intrv(scm_cp, cols, ind, obs)
                    intv_dict_causes = {k: intv_dict[k] for k in acs & intv_dict.keys()}
                    scm_cp = scm_cp.fix_nondescendants(intv_dict_causes, obs)
                    scm_cp.sample_context(SUBPOPULATION_SIZE)
                    data = scm_cp.compute(do=intv_dict)

                    # compute acceptance rate
                    pred_proba = np.exp(model.predict_log_proba(data.loc[:, cols]))[:, 1]
                    mean_proba = np.mean(pred_proba)
                    acceptance_rate = np.mean(pred_proba >= thresh)

                    if use_scm_pred:
                        # use scm_prediction
                        preds_scm = []
                        for kk in range(len(data)):
                            pred_proba_scm = scm.predict_log_prob_obs(data.iloc[kk, :], scm.predict_target)
                            preds_scm.append(np.exp(pred_proba_scm))
                        preds_scm = np.array(preds_scm)

                        # compute acceptance rate
                        mean_proba_scm = np.mean(preds_scm)
                        acceptance_rate_acm = np.mean(preds_scm > thresh)


                    # compile to observation
                    res_model = {'gamma': meaningfulness, 'eta_emp': acceptance_rate,
                                 'mean_proba': mean_proba, 'type': 'model', 'gamma_spec': gamma}
                    res_model = dict(res_model, **intv_dict_causes)

                    res = pd.Series(res_model)
                    df_res_ind.append(res)


                    if use_scm_pred:
                        res_scm = {'gamma': meaningfulness, 'eta_emp': acceptance_rate_acm,
                                   'mean_proba': mean_proba_scm, 'type': scm_name, 'gamma_spec': gamma}
                        res_scm = dict(res_scm, **intv_dict_causes)
                        res_scm = pd.Series(res_scm)
                        df_res_ind.append(res_scm)

                df_res_ind = pd.concat(df_res_ind, axis=1, ignore_index=True).T
                df_res_ind['id'] = jj
                df_res.append(df_res_ind)

        df_res = pd.concat(df_res, ignore_index=True)

        results[scm_name] = df_res

    results[scm_name] = pd.concat([results[scm_name], df_res], ignore_index=True)

    import seaborn as sns
    import matplotlib.pyplot as plt

    def get_bound(gamma, thresh):
        return (gamma - thresh) / (1 - thresh)

    def convert_data(df, except_cols=None):
        if except_cols is None:
            cols = df.columns
        else:
            ls = list(df.columns)
            for col in except_cols:
                if col in ls:
                    ls.pop(ls.index(col))
            cols = ls
        for col in cols:
            df[col] = np.array(df[col], dtype=np.float64)
        return df

    df_all = []
    for scm in scms:
        df = convert_data(results[scm], except_cols=['type', 'eta_emp_type', 'scm'])
        df_2 = df.copy()
        df_2['eta_emp'] = get_bound(df['gamma_spec'], thresh)
        df['eta_emp_type'] = 'emp'
        df_2['eta_emp_type'] = 'bound'
        df = pd.concat([df, df_2])
        df['scm'] = scm
        df_all.append(df)
    df_all = pd.concat(df_all, ignore_index=True)

    results[scm_name].to_csv(savepath + 'results.csv')

    sns.lineplot(data=df_all.reset_index(), x='gamma_spec', y='eta_emp',
                 hue='eta_emp_type', style='scm')
    plt.savefig(savepath + 'lineplot.pdf')
    plt.show()

    df_all.to_csv(savepath + 'df_all.csv')'''